In [1]:
# import libraries
import json
import pandas as pd
import re
import time
import os
import geopandas as gpd
from IPython.display import display
import googlemaps
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopy
from geopy.exc import GeocoderTimedOut

# Clean property features

In [2]:
# read in property json file
property_df = pd.read_json("../data/raw/property.json").transpose().reset_index()

# rename columns
property_df.columns = ['url', 'address', 'rent', 'features', 'type', 'furnitured', 'pool', 'gym', 'coordinates','desc']

# drop url column
property_df = property_df.drop(columns = ['url'])

In [3]:
# drop duplicate properties
unique_property_df = property_df.iloc[property_df.astype(str).drop_duplicates(subset=None, keep='first', inplace=False).index]

# index for graph
# property_df["index"] = property_df.index.to_list()

In [4]:
'''Get latitue from coordinate'''
def latitude(coord):
    return coord[0]

'''Get longtitude from coordinate'''
def longtitude(coord):
    return coord[1]

In [5]:
# put latitude and longititude information in dataframe
property_df["Latitude"] = property_df['coordinates'].apply(latitude)
property_df["Longtitude"] = property_df['coordinates'].apply(longtitude)

In [6]:
# check the structure of property data
unique_property_df.head()

,address,rent,features,type,furnitured,pool,gym,coordinates,desc
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,$400 Per Week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,"[-37.8102832, 144.9566691]",South Melbourne Leasing
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,$350 per week,1 Bed1 Bath− Parking,Studio,Yes,No,No,"[-37.810779, 144.9685513]",Wimpie Santoso
2,911/408 Lonsdale Street Melbourne VIC 3000,$330 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,"[-37.8125979, 144.9604012]",Ender Gok
3,918/422 Collins St Melbourne VIC 3000,$600 Per week fully furnished,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,"[-37.8170971, 144.9601487]",Precinct Leasing
4,602/118 Franklin Street Melbourne VIC 3000,$330,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,"[-37.8082052, 144.9589035]",Manuel Flores


In [7]:
# check and drop duplicate properties
unique_property_df.shape

(15075, 9)

In [8]:
# get postcode for all properties from the address
unique_property_df["postcode"] = unique_property_df["address"].\
                                apply(lambda x: int(x[-4:]))

C:\Users\dell\AppData\Local\Temp/ipykernel_28264/3942963013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_property_df["postcode"] = unique_property_df["address"].\


## Combine external dataset and property data

In [9]:
# reverse the coordinates to address
def reverse_coord_geopy(coordinates):
    #access server
    geopy.geocoders.options.default_user_agent = "my"
    geolocator = Nominatim(user_agent="my")
    try:
        reverse = RateLimiter(geolocator.reverse, min_delay_seconds=2)
        location=reverse(coordinates,language='en',exactly_one=True, timeout=60)  #find location
        address = location.raw["address"]   #extract address
        return address
    except GeocoderTimedOut as e:    #if meet time out, wait for 2 second and reverse again
        time.sleep(2)
        return reverse_coord_geopy(coordinates)
    
def generate_suburb_postcode_geopy(df):
    #reverse
    df["Reverse_Coord"]= df["coordinates"].apply(reverse_coord_geopy)
    return df

In [11]:
# reverse geolocation and get address
property_address = pd.DataFrame()
# reverse the coordinates by small size due to request limitation
i = 0
while i < 16000:
    if i < 15000:
        property_address_i = generate_suburb_postcode_geopy(property_df.copy().iloc[i:i+1000])
        property_address = pd.concat([property_address,property_address_i],ignore_index=True)
    else:
        property_address_i = generate_suburb_postcode_geopy(property_df.copy().iloc[i:])
        property_address = pd.concat([property_address,property_address_i],ignore_index=True)
    print(i)
    i+=1000
# save GNR Data
path = os.getcwd().replace("notebooks","") + "data/curated/"
property_address.to_csv(path+'property_suburb.csv',index=False)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


## Get bed number, bath number and car park number for each property

In [29]:
# read in combined dataset
clean_features = pd.read_csv("../data/curated/property_suburb.csv")

In [30]:
clean_features.shape

(15117, 12)

In [31]:
# drop the property with empty features
clean_features = clean_features[clean_features["features"] != ""].reset_index(drop=True)
clean_features = clean_features.dropna(how='any')

# check the size of the data
clean_features.shape

(15083, 12)

In [34]:
num_bed_list = []
num_bath_list = []
num_park_list = []

# split the detail from the property features
for i in clean_features["features"]:

    # find the number of bedroom, bathroom and parking area
    num_bed = re.findall(r'(\d|-|−)\s?Bed', i)
    num_bath = re.findall(r'(\d|-|−)\s?Bath', i)
    num_park = re.findall(r'(\d|-|−)\s?Park', i)

    # if no information, set the number to zero
    if (len(num_bed) == 0) or (num_bed[0].isdigit() == False):
        num_bed = ["0"]
    if (len(num_bath) == 0) or (num_bath[0].isdigit() == False):
        num_bath = ["0"]
    if (len(num_park) == 0) or (num_park[0].isdigit() == False):
        num_park = ["0"]
    
    # append the data together
    num_bed_list.append(int(num_bed[0]))
    num_bath_list.append(int(num_bath[0]))
    num_park_list.append(int(num_park[0]))

# generate the new features of the property
clean_features["num_bed"] = num_bed_list
clean_features["num_bath"] = num_bath_list
clean_features["num_park"] = num_park_list

In [35]:
# drop the property with no bedroom
clean_features = clean_features[clean_features["num_bed"] != 0].reset_index(drop=True)

# Clean property rent

In [37]:
# read in the csv file after first cleaned property features
# path = "../data/curated/cleaned_property_with_features.csv"
# property_data = pd.read_csv(path)
property_data = clean_features.copy()

In [38]:
# check the shape of property data
property_data.shape

(14932, 15)

In [39]:
# get the rent price data
rent_df_new = pd.DataFrame()
rent_df_new["rent"] = property_data["rent"]

# clean the rent data to gain the weekly rent
for i in range(len(rent_df_new["rent"])):

    # get the price
    price = rent_df_new["rent"][i].lower()
    rent = re.findall(r"[$](\d+\.?,?\d*)", price)

    # find the annually rent
    if ("p.a." in price) or ("annually" in price) or ("pa." in price):
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))/52

    # find the fortnightly rent
    elif "fortnightly" in price:
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))/2

    # find the nightly rent
    elif "night" in price:
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)
        rent_df_new["rent"][i] = float(rent[0].replace(",", "")) * 7

    # find the monthly rent
    elif ("pcm" in price) or ("pm" in price) or ("month" in price):

        # check whether contain weekly rent
        if re.findall(r"(p([a-z]*|\/*)\s?w)", price) != "":
            if len(rent) == 0:
                rent = re.findall(r"(\d+\.?,?\d*)", price)
            rent_df_new["rent"][i] = float(rent[0].replace(",", ""))
            
        # if only contain monthly rent, calculate the weekly rent
        else:
            if len(rent) == 0:
                rent = re.findall(r"(\d+\.?,?\d*)", price)
            rent = (float(rent[0].replace(",", "")) / 30) * 7
            rent_df_new["rent"][i] = rent
        
    # find the weekly rent
    else:
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)

            # set the rent without any price data as zero
            if len(rent) == 0:
                rent_df_new["rent"][i] = 0
            else:
                rent_df_new["rent"][i] = float(rent[0].replace(",", ""))
        
        elif len(rent) == 2:
            num1 = int(float(rent[0].replace(",", "")))
            num2 = int(float(rent[1].replace(",", "")))

            # check for normal weekly rent range
            if len(str(num1)) != len(str(num2)):
                # check whether the larger price is twice than the lower price, 
                # if yes, using lower price, if no, using the mean price
                if num1 * 2 < num2:
                    rent = num1
                else:
                    rent = (num1 + num2)/2
            else:
                rent = (num1 + num2)/2
            rent_df_new["rent"][i] = rent

        else:
            rent_df_new["rent"][i] = float(rent[0].replace(",", ""))


# transfer the rent data type to integer
rent_df_new = rent_df_new.astype(int)

In [40]:
# check the shape
rent_df_new.shape

(14932, 1)

In [41]:
# add the processed rent data into the whole data set
property_data["rent_weekly"] = rent_df_new["rent"]

In [42]:
# check the shape of the prperty data
property_data.shape

(14932, 16)

In [43]:
# clean the zero rent
property_data = property_data[property_data["rent_weekly"] != 0]
property_data = property_data.reset_index()

In [44]:
# check the dataframe shape after clean the zero rent
property_data.shape

(14564, 17)

# Clean property floor

In [52]:
# rename the dataframe to clean floor information
new_property_df = property_data.copy()

# initial number of the floor is -1
floor = []
for i in range(len(new_property_df["address"])):
    floor.append('-1')

# create the floor feature
new_property_df["floor"] = floor

In [53]:
new_property_df.shape

(14564, 18)

In [47]:
for i in range(len(new_property_df["address"])):
    # change the building that only have one floor only (such as Villa)
    if (new_property_df["type"][i] == "House") or (new_property_df["type"][i] == "Villa") or (new_property_df["type"][i] == "New House & Land") or (new_property_df["type"][i] == "Semi-Detached") or (new_property_df["type"][i] == "Vacant land"):
        new_property_df.loc[[i],['floor']]  = 1

    elif (new_property_df["type"][i] == "Rural") or (new_property_df["type"][i] == "Farm"):
        new_property_df.loc[[i],['floor']]  = 1
    else:
        location = new_property_df["address"][i].split(" ")[0]
        if "/" in location:  
            floor_str = location.split("/")
            # create temporary variables to store the floor information
            f=''
            fff=''
            # process the floor address is not digital.
            if floor_str[0].isdigit() == False: 
                # special case
                # ground floor
                if ("G" in floor_str[0]) or ("Ground" in floor_str[0]):
                    new_property_df.loc[[i],['floor']]  = 1
                if ("LG" in floor_str[0]):
                    new_property_df.loc[[i],['floor']]  = 1
                # switch the format of the floor address from various form to uniform digitals.
                if ("Level" in floor_str[0]) or ("L" in floor_str[0]):  
                    if len(re.findall('[0-9]', floor_str[0])) != 0:
                        new_property_df.loc[[i],['floor']] = int(f.join(re.findall("[0-9]", floor_str[0])))
                elif ("&" in floor_str[0]):
                    new_property_df.loc[[i],['floor']]  = floor_str[0][0]
                elif ("." in floor_str[0]):
                    if ("AP" in floor_str[0]):
                        new_property_df.loc[[i],['floor']]  = floor_str[0].split(".")[0][-1] 
                    else:
                        if len(re.findall('[0-9]', floor_str[0].split('.')[0])) != 0:
                            new_property_df.loc[[i],['floor']]  = int(fff.join(re.findall('[0-9]', floor_str[0].split('.')[0])))
                else:
                    # Determine the floor number when there is a letter on the end of the floor address such as '1014B'
                    if re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]) != None:
                        tmp_floor = re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]).group()[:-1]
                        if len(tmp_floor) == 1 or len(tmp_floor) == 2:
                            new_property_df.loc[[i],['floor']] = tmp_floor
                        elif len(tmp_floor) == 3 and int(tmp_floor[1]) > 1:
                            new_property_df.loc[[i],['floor']] = tmp_floor[:2]
                        elif len(tmp_floor) == 3 and int(tmp_floor[1]) <= 1:
                            new_property_df.loc[[i],['floor']] = tmp_floor[0]
                        elif len(tmp_floor) == 4:
                            new_property_df.loc[[i],['floor']] = tmp_floor[:2]       
                                        
            # determine which number is the floor. for example: 319 is 3th floor and 3109 is the 31th floor.
            else:
                if len(floor_str[0]) == 1 or len(floor_str[0]) == 2:
                    new_property_df.loc[[i],['floor']]  = floor_str[0]
                elif len(floor_str[0]) == 3 and int(floor_str[0][1]) > 1:
                    new_property_df.loc[[i],['floor']]  = floor_str[0][:2]
                elif len(floor_str[0]) == 3 and int(floor_str[0][1]) <= 1:
                    new_property_df.loc[[i],['floor']]  = floor_str[0][0]
                elif len(floor_str[0]) == 4:
                    new_property_df.loc[[i],['floor']]  = floor_str[0][:2]

        # process the address without '/'
        else: 
            # check the floors that have "," on the tail.       
            if ',' in new_property_df["address"][i]:
                floor_str111 = new_property_df["address"][i].split(",")[0]
                if len(floor_str111.split(' '))>1:
                    FL = floor_str111.split(' ')[0]
                    FF = floor_str111.split(' ')[1]
                    if ("Level" in FL) or ("Lvl" in FL):  
                        new_property_df.loc[[i],['floor']] = FF
            # Determine the floor number when there is a letter on the end of the floor address such as '1014B'
            if re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]) != None:
                tmp_floor = re.search('(\\d+)[A-Za-z]',new_property_df["address"][i]).group()[:-1]
                if len(tmp_floor) == 1 or len(tmp_floor) == 2:
                    new_property_df.loc[[i],['floor']]  = tmp_floor
                elif len(tmp_floor) == 3 and int(tmp_floor[1]) > 1:
                    new_property_df.loc[[i],['floor']]  = tmp_floor[:2]
                elif len(tmp_floor) == 3 and int(tmp_floor[1]) <= 1:
                    new_property_df.loc[[i],['floor']]  = tmp_floor[0]
                elif len(tmp_floor) == 4:
                    new_property_df.loc[[i],['floor']]  = tmp_floor[:2]  

In [ ]:
count=0
# test the results that has no floor information
for i in range(len(new_property_df["address"])):
    location = new_property_df["address"][i].split('/')[0]
    if new_property_df["floor"][i] == '-1':
        
        print(new_property_df["address"][i])
        print(new_property_df["floor"][i])
        count+=1
print("Count of the address without floor information: ", count)
# check the shape of the cleaned dataframe
new_property_df.shape

In [51]:
# save the new dataframe to csv
new_property_df.to_csv('../data/curated/cleaned_property_data.csv', index = None)